In [3]:

!pip install evaluate datasets transformers rouge_score bert_score



  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [1]:
import os
from evaluate import evaluator
from datasets import load_dataset
from transformers import pipeline, BartTokenizer, PegasusTokenizer, AutoTokenizer
from bert_score import score as bertscore_score
import time

In [61]:
# Define model names and their maximum sequence lengths
model_configs = {
    "facebook/bart-large-cnn",  # BART model typically has a max length of 1024
    "google/pegasus-xsum",       # Pegasus model has a max length of 512
    "falconsai/text_summarization"  # Assuming Falcon model also has a max length of 512
}

# Define generation parameters
generation_kwargs = {
    "max_length": 90,  # Set a maximum length for the generated summary
    "min_length": 30,  # Set a minimum length for the generated summary
    "length_penalty": 2.0,
    "no_repeat_ngram_size": 3,
    "early_stopping": True
}



In [48]:
import pandas as pd
from datasets import load_dataset, Dataset

# Load the dataset
data1 = load_dataset("cnn_dailymail", "3.0.0", split="test[:350]")  # Adjust the size as needed

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset)

# Define maximum article length in characters
max_article_length = 2000

# Filter out rows where the length of the article exceeds the maximum length
filtered_df = df[df['article'].str.len() <= max_article_length]

# Convert the filtered DataFrame back to a Dataset
dataset = Dataset.from_pandas(filtered_df)

# Print the number of remaining examples
print(f"Number of examples after filtering: {len(filtered_dataset)}")

# Optionally: Save the filtered dataset to a CSV file
# filtered_df.to_csv("filtered_dataset.csv", index=False)
df = pd.DataFrame(dataset)

Number of examples after filtering: 106


In [53]:
dataset

Dataset({
    features: ['article', 'highlights', 'id', '__index_level_0__'],
    num_rows: 106
})

In [56]:
import pandas as pd

# Assuming you have a DataFrame named 'df' with a column 'article'
max_length_index = df['article'].str.len().idxmax()
max_length_article = df.loc[max_length_index, 'article']

print("Article with the longest length:")
print(f"Index: {max_length_index}")
print(max_length_article)
print("Length:", len(max_length_article))

Article with the longest length:
Index: 95
Tokyo (CNN)A second robotic probe sent into the crippled Fukushima nuclear plant has captured images of a strange green glow. Tokyo Electric Power Company (TEPCO) deployed the second remote-controlled robot last week after the first one broke down. The robot detected lower radiation levels and temperature than expected, an indicator that cooling systems were working effectively, according to a statement released by TEPCO. "It is a great step forward towards the decommissioning work as we can earn necessary data for the next investigation," said Akira Ono, the chief of Fukushima Daiichi plant. TEPCO said the yellow seen on the images seemed to suggest a discoloration of the grating, though the cause was unknown. It said the green glow could not be seen when filmed from other angles. The shape-shifting robots were sent in to assess the damage in one of the reactors that suffered a meltdown after a devastating earthquake and tsunami in March 2011

In [57]:
from transformers import AutoTokenizer
text= df.article[95]
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize(text)

print(len(tokens))

412


In [58]:
import pandas as pd

# Assuming you have a DataFrame named 'df' with a column 'article'
max_length_index = df['highlights'].str.len().idxmax()
max_length_article = df.loc[max_length_index, 'highlights']

print("Article with the longest length:")
print(f"Index: {max_length_index}")
print(max_length_article)
print("Length:", len(max_length_article))

Article with the longest length:
Index: 60
Marvel Studios releases first looks at Paul Bettany as the Vision in "Avengers: Age of Ultron" and Charlie Cox in full "Daredevil" costume .
Jamie Bell's character of The Thing was also unveiled for 20th Century Fox's Marvel-based reboot of "Fantastic Four"
Bryan Singer unveiled the first look at "X-Men: Apocalypse" Angel played by Ben Hardy .
Length: 345


In [59]:
from transformers import AutoTokenizer
text= df.highlights[60]
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize(text)

print(len(tokens))

81


In [64]:
import os
from evaluate import evaluator
from datasets import load_dataset
from transformers import pipeline, BartTokenizer, PegasusTokenizer, AutoTokenizer
from bert_score import score as bertscore_score
import time

# Define model names and their maximum sequence lengths
model_configs = {
    "facebook/bart-large-cnn": 1024,  # BART model typically has a max length of 1024
    "google/pegasus-xsum": 512,       # Pegasus model has a max length of 512
    "falconsai/text_summarization": 512  # Assuming Falcon model also has a max length of 512
}

# Define generation parameters
generation_kwargs = {
    "max_length": 90,  # Set a maximum length for the generated summary
    "min_length": 30,  # Set a minimum length for the generated summary
    "length_penalty": 2.0,
    "no_repeat_ngram_size": 3,
    "early_stopping": True
}

# Define maximum input length in tokens
max_input_tokens = 512



# Collect predictions and references
for model_name, model_max_length in model_configs.items():
    print(f"Processing model: {model_name}")

    # Initialize the model pipeline for summarization
    device = -1  # Use CPU
    summarizer = pipeline("summarization", model=model_name, device=device)

    # Initialize tokenizer based on model
    if model_name == "facebook/bart-large-cnn":
        tokenizer = BartTokenizer.from_pretrained(model_name)
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Reset lists for predictions and references
    predictions = []
    references = []

    # Start processing
    start_time = time.time()
    total_examples = len(dataset)
    print(f"Processing {total_examples} examples...")

    for i, example in enumerate(dataset):
        try:
            article = example["article"]
            highlight = example["highlights"]

            # Tokenize the article to check its length
            tokenized_input = tokenizer(article, return_tensors="pt", truncation=False)
            input_length = tokenized_input['input_ids'].size(1)  # Length in tokens

            # Check if the article length is within the acceptable limit
            if input_length <= max_input_tokens:
                # Truncate or split the article to ensure it fits within the model's max length
                if input_length > model_max_length:
                    print(f"Truncating article {i+1}/{total_examples} to fit model's max length")
                    article = tokenizer.decode(tokenized_input['input_ids'][0][:model_max_length], skip_special_tokens=True)
                else:
                    article = tokenizer.decode(tokenized_input['input_ids'][0], skip_special_tokens=True)

                # Generate a summary for the article
                summary_start_time = time.time()
                summary = summarizer(article, **generation_kwargs)
                summary_end_time = time.time()

                generated_summary = summary[0]['summary_text']

                # Append to lists
                predictions.append(generated_summary)
                references.append(highlight)

                # Print progress
                print(f"Processed example {i+1}/{total_examples} in {summary_end_time - summary_start_time:.2f} seconds")

            else:
                print(f"Skipped example {i+1}/{total_examples} due to excessive length")

        except Exception as e:
            print(f"Error processing example {i+1}: {e}")

    # Compute BERTScore
    if predictions and references:
        try:
            bertscore_start_time = time.time()
            precision, recall, f1 = bertscore_score(predictions, references, lang="en")
            bertscore_end_time = time.time()

            # Print BERTScore results in a simplified format
            print("Results for BERTScore:")
            print(f"Precision: {precision.mean():.4f} (Mean Precision)")
            print(f"Recall: {recall.mean():.4f} (Mean Recall)")
            print(f"F1 Score: {f1.mean():.4f} (Mean F1 Score)")
            print(f"BERTScore computation time: {bertscore_end_time - bertscore_start_time:.2f} seconds")

        except Exception as e:
            print(f"Error computing BERTScore: {e}")
    else:
        print("No predictions or references collected for BERTScore.")

    # Optionally, evaluate with other metrics using SummarizationEvaluator
    task_evaluator = evaluator("summarization")
    metrics = ["rouge", "bleu", "meteor"]

    for metric in metrics:
        try:
            metric_start_time = time.time()
            results = task_evaluator.compute(
                model_or_pipeline=model_name,
                data=dataset,
                metric=metric,  # Evaluate one metric at a time
                input_column="article",  # Column containing the text to summarize
                label_column="highlights",  # Column containing the reference summaries
                generation_kwargs=generation_kwargs,
            )
            metric_end_time = time.time()

            print(f"Results for {metric}:")
            print(results)
            print(f"{metric} computation time: {metric_end_time - metric_start_time:.2f} seconds")

        except Exception as e:
            print(f"Error evaluating metric {metric}: {e}")

    # Print total processing time for the model
    end_time = time.time()
    print(f"Total processing time for model {model_name}: {end_time - start_time:.2f} seconds")
    print("-" * 50)


Processing model: facebook/bart-large-cnn


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Processing 106 examples...
Processed example 1/106 in 4.46 seconds
Processed example 2/106 in 5.70 seconds
Processed example 3/106 in 3.97 seconds
Processed example 4/106 in 3.05 seconds
Processed example 5/106 in 4.67 seconds
Processed example 6/106 in 6.15 seconds
Processed example 7/106 in 5.41 seconds
Processed example 8/106 in 3.94 seconds
Processed example 9/106 in 5.73 seconds
Processed example 10/106 in 3.38 seconds
Processed example 11/106 in 4.91 seconds
Processed example 12/106 in 4.17 seconds
Processed example 13/106 in 5.94 seconds
Processed example 14/106 in 3.75 seconds
Processed example 15/106 in 5.85 seconds
Processed example 16/106 in 2.96 seconds
Processed example 17/106 in 4.19 seconds
Processed example 18/106 in 4.89 seconds
Processed example 19/106 in 2.58 seconds
Processed example 20/106 in 4.84 seconds
Processed example 21/106 in 4.03 seconds
Processed example 22/106 in 4.95 seconds
Processed example 23/106 in 4.45 seconds
Processed example 24/106 in 3.28 second

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results for BERTScore:
Precision: 0.8713 (Mean Precision)
Recall: 0.8860 (Mean Recall)
F1 Score: 0.8785 (Mean F1 Score)
BERTScore computation time: 118.45 seconds
Results for rouge:
{'rouge1': 0.35102283845950777, 'rouge2': 0.15312021236354018, 'rougeL': 0.2567426031773986, 'rougeLsum': 0.29726931178779775, 'total_time_in_seconds': 500.39491149199966, 'samples_per_second': 0.21183268967293392, 'latency_in_seconds': 4.720706712188676}
rouge computation time: 505.47 seconds
Results for bleu:
{'bleu': 0.11708366102114602, 'precisions': [0.3246261216350947, 0.133224689346099, 0.08099104726212783, 0.05365126676602087], 'brevity_penalty': 1.0, 'length_ratio': 1.3842119790229093, 'translation_length': 5015, 'reference_length': 3623, 'total_time_in_seconds': 493.8022543980005, 'samples_per_second': 0.21466082638530215, 'latency_in_seconds': 4.658511833943401}
bleu computation time: 499.05 seconds


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Results for meteor:
{'meteor': 0.35965923423093993, 'total_time_in_seconds': 501.9880890730019, 'samples_per_second': 0.21116038867723988, 'latency_in_seconds': 4.7357366893679425}
meteor computation time: 505.95 seconds
Total processing time for model falconsai/text_summarization: 2121.83 seconds
--------------------------------------------------


In [65]:
result

NameError: name 'result' is not defined